In [1]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)

import cspython.analysis as a
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
#import sklearn
#print('sklearn: {}'.format(sklearn.__version__))
#import xlrd

import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline


import sklearn
print('sklearn: {}'.format(sklearn.__version__))
#import xlrd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
#import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
import seaborn as sns
from sklearn import preprocessing 
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

scipy: 0.19.0
numpy: 1.11.3
matplotlib: 2.1.0
pandas: 0.22.0
sklearn: 0.18.1


C:\Users\j_coo\Anaconda2new\envs\csgoproject\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
with open('../cspython/test.pkl', 'rb') as f: 
     d = pkl.load(f)

In [7]:
big_data = process_scrapped(d)
overview, big_data = big_data

In [10]:
big_data.keys()

['e66048a5-8a32-477d-bae6-aea235e1b50e',
 '31e7fb31-a1f4-40f6-9f42-3652d95438cf',
 '4042df67-b86c-4583-abdf-ceb3c5b7ae31',
 'a6c9a66d-1a36-4c36-9ee2-cf85abd5210b',
 'c3e9af64-f0ff-448c-ad04-6a57e0480aab',
 '9515fdcb-472c-4590-8941-6cb9decd7c8c',
 '0d9622b1-db84-4698-b386-f52decfa9585',
 'b7c6b7d9-f9ca-4f49-9a2f-348e1236ed4f',
 'e9cab27e-26b7-4e17-bafe-380d6d45a315',
 'fbe1812a-08fc-4a05-b4ee-1929591b7fb2']

In [17]:
key_being_used = big_data.keys()[0]

In [19]:
big_data[key_being_used]['match_data']

[{'awp_kills':   Unnamed: 0  yay dephh ANDROID-X23  ptr  FNS
  0        ryx  0:0   0:1         0:0  0:1  0:0
  1       Hiko  0:0   0:2         0:0  0:2  0:0
  2       SicK  1:0   2:2         0:0  0:3  0:0
  3       vice  0:0   0:0         0:0  0:2  0:0
  4     cadiaN  0:0   0:1         1:0  0:2  0:0,
  'first_kills':   Unnamed: 0  yay dephh ANDROID-X23  ptr  FNS
  0        ryx  1:0   0:0         1:2  0:1  0:0
  1       Hiko  1:0   1:1         0:1  1:0  0:0
  2       SicK  1:0   1:1         0:0  0:1  0:1
  3       vice  0:0   0:0         0:1  3:0  1:0
  4     cadiaN  1:0   1:0         1:0  0:1  0:1,
  'total_team_kda':   Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3
  0      Kills          90         NaN          70
  1     Deaths          70         NaN          90
  2    Assists          13         NaN          14,
  'who_kill_who':   Unnamed: 0  yay dephh ANDROID-X23  ptr  FNS
  0        ryx  4:3   1:7         5:4  1:3  1:2
  1       Hiko  2:4   3:5         0:4  2:2  6:4
  2       S

In [ ]:
def combine_dfs(big_data):
    for l in range(len(big_data.keys())):
        series_data = big_data[big_data.keys()[l]]
        series_data_m = merge_matches(series_data)
        series_data_mo = merge_overview(series_data_m, series_data)
        series_data_mos = merge_scoreboards(series_data_mo, series_data)
        if l == 0:
            data = series_data_mos
        else:
            data = pd.concat([data, series_data_mos])
    return data

In [ ]:
def merge_matches(series_data):
    for d in range(0,len(series_data['matches'])):
        if d == 0:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = series_data['matches'][d]
        else:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = pd.concat([series_data_m, series_data['matches'][d]])
    return series_data_m
    #should work for concact the matches together


In [ ]:
def merge_overview(series_data_m, series_data):
    series_data['match_overview'] = series_data['match_overview'].rename(index = str, columns ={series_data['match_overview'].columns[4]: "team_A_score",series_data['match_overview'].columns[5]: "team_B_score"})
    series_data_mo = pd.merge(series_data_m, series_data['match_overview'], on=['match_id', 'winner', 'map', 'series_id'])
    return series_data_mo
#works at merging matches with match_overviewb

In [ ]:
def merge_scoreboards(series_data_mo, series_data):    
    for i in range(len(series_data['scoreboards'][0])):
        series_data['scoreboards'][0][i]['match_num'] = i+1
        series_data['scoreboards'][0][i] = series_data['scoreboards'][0][i].rename(index = str, columns={ series_data['scoreboards'][0][i].columns[0] : "team_players" })
        series_data['scoreboards'][1][i]['match_num'] = i+1
        series_data['scoreboards'][1][i] = series_data['scoreboards'][1][i].rename(index = str, columns={ series_data['scoreboards'][1][i].columns[0] : "team_players"})
        new_df = pd.concat([series_data['scoreboards'][0][i], series_data['scoreboards'][1][i]])
        if i == 0:
            con_df = new_df
        else:
            con_df = pd.concat([con_df, new_df])
    series_data_mos = pd.merge(series_data_mo, con_df, how='outer', on='match_num') 
    return series_data_mos

       
        
#works at adding match_num to scoreboards
        
                         

In [ ]:
data = combine_dfs(big_data)

In [ ]:
data.shape

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data

In [ ]:
data = pd.concat([data,data['K-D'].str.split('-', 1, expand=True).rename(columns={0:'player_kill', 1:'player_death'})], axis=1)


In [ ]:
data.shape

In [ ]:
data = data.apply(pd.to_numeric, errors='ignore')

In [ ]:
data[['map', 'ending', 'CT', 'T', 'side_winner','winner', 'team_players']]= data[['map', 'ending', 'CT', 'T', 'side_winner','winner', 'team_players']].apply(lambda x: pd.factorize(x)[0]) 

In [ ]:
data['KAST'] = data['KAST'].apply(lambda x : float(x.strip('%')) / 100.0)

In [ ]:
data.dtypes

In [ ]:
corr = data.select_dtypes(include = ['float64', 'int64', 'int32']).iloc[:, 0:].corr()
plt.figure(figsize=(25, 25))
sns.heatmap(corr, vmax=1, square=True)



In [ ]:
ydata = data['winner']

In [ ]:
data = data.drop('winner', 1)

In [ ]:
data = data.drop('match_id', 1)

In [ ]:
data = data.drop('series_id', 1)

In [ ]:
data = data.drop('K-D', 1)

In [ ]:
data.shape

In [ ]:
y = ydata.values
X = data.values[:,0:19]

In [ ]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'

In [ ]:
def local_cv(model, params):                             #KFOLD WITH GRID SEARCH
    param_grid = params
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid_result = grid.fit(X, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for params, mean_score, scores in grid_result.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [ ]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'
model = GradientBoostingClassifier()
params = {
 'max_depth':[4,6,8,10,12,16],
 'n_estimators':[20,30,50,80,100],
'learning_rate':[.1,.01,.001]}
local_cv(model, params)

In [ ]:
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('NB', GaussianNB()))
#models.append(('CART', DecisionTreeClassifier())) 
#models.append(('KNeighborsClassifier', KNeighborsClassifier())) 
#models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
#models.append(('AdaBoostClassifier', AdaBoostClassifier()))
#models.append(('RandomForestClassifier', RandomForestClassifier()))
#models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('SVC', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    print '1'
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    print '2'
    cv_results = cross_validation.cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    print '3'
    results.append(cv_results)
    print '4'
    names.append(name)
    print '5'
    model.fit(X, y)
    print '6'
    predicted = model.predict(X)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)